# MA 195L.2 Programming Assignment 1

### Instructor: Jakov Ivan S. Dumbrique (jdumbrique@ateneo.edu)

MA 195L.2: Introduction to Financial Mathematics II \
Second Semester, S.Y. 2020-2021 \
Ateneo de Manila University

Submitted by: Brian Godwin S. Lim


Websites I used for completing this PA:

https://community.insaid.co/hc/en-us/articles/360052303453-How-to-read-a-csv-dataset-from-GitHub-in-Pandas-

https://www.w3resource.com/python-exercises/pandas/python-pandas-data-frame-exercise-65.php

## Part 1

Import the necessary packages

In [2]:
import numpy as np # Numerical Computing
import pandas as pd # Data Wrangling
import matplotlib.pyplot as plt # Plotting
from scipy.stats import norm # Statistical Analysis
from statistics import NormalDist # Statistical Analysis

%matplotlib inline

Define the get_return and get_weight function

In [3]:
def get_return(df):
    return np.log(df["close"]/df["previous"])

def get_weights(df,d,ewma_par):
    count_returns = len(df["close"])-d
    weight_lst = [(1-ewma_par)*(ewma_par**i) for i in range(count_returns)]
    df["weight"] = pd.Series(weight_lst)
    return df

Define the undiversified_d_day_p_percent_VaR_delta_normal

In [4]:
def undiversified_d_day_p_percent_VaR_delta_normal(
    df_lst, N_lst, d, p, 
    ewma_par_lst=None
):
    """Returns the undiversified d-day p% VaR of a portfolio of stocks using Delta Normal Approach.
    
    Parameters
    ----------
    df_lst : list of pandas.DataFrame
        each df has two columns: (1) dt [str] and (2) closing price [float]
        assumes the dates are arranged from newest to oldest, and the date today is the date on the first row
        asssumes that all dfs have the same ordered list of dates
        there is one df per stock in the portfolio
    N_lst : list (of int)
        N_i = number of shares for ith stock (order should be the same as the order in df_lst)
    d : int
        the value to be used in calculating the d-day VaR (e.g. 1-day, 5-day)
    p : int
        the value to be used in calculating the p% VaR (e.g. 99, 95)
    ewma_par_lst : list (of float)
        ewma_par_i = the value of the lambda parameter in an exponentially-weighted moving average model 
        for ith stock (order should be the same as the order in df_lst) 
        assumes the value is in range (0,1)
        assumes there is no None value in the list
    
    Returns
    -------
    float (2 decimal places)
        undiversified d-day p% VaR of a portfolio of stocks using Delta Normal Approach
    """
    
    # Initialize return value 
    VaR = 0 
    
    # Process each df in df_lst
    for i in range(len(df_lst)): 
        
        # Get the closing price
        s0 = df_lst[i].loc[0,"close"] 
                
        # Get the return
        df_lst[i]["previous"] = df_lst[i]["close"].shift(-d)
        df_lst[i]["return"] = df_lst[i].apply(get_return,axis=1)     
        
        if ewma_par_lst:
            # Solve for the weights and compute the standard deviation
            df_lst[i] = get_weights(df_lst[i],d,ewma_par_lst[i])
            sigma = np.sqrt(np.nansum((df_lst[i]["return"]**2) * df_lst[i]["weight"]))
            
        else:
            # Get the standard deviation
            sigma = df_lst[i]["return"].std() 
        
        quantile = NormalDist().inv_cdf(p/100)
        
        # Check the variations for Part 2 portfolio C
        #print(s0*N_lst[i],sigma)
        
        VaR += s0 * N_lst[i] * sigma * quantile
    
    return VaR

Defining the diversified_d_day_p_percent_VaR_delta_normal

In [5]:
def diversified_d_day_p_percent_VaR_delta_normal(
    df_lst, N_lst, d, p, 
    ewma_par_lst=None,
    covar_ewma_par_mat=None
):
    """Returns the diversified d-day p% VaR of a portfolio of stocks using Delta Normal Approach.
    
    Parameters
    ----------
    df_lst : list of pandas.DataFrame
        each df has two columns: (1) dt [str] and (2) closing price [float]
        assumes the dates are arranged from newest to oldest, and the date today is the date on the first row
        asssumes that all dfs have the same ordered list of dates
        there is one df per stock in the portfolio
    N_lst : list (of int)
        N_i = number of shares for ith stock (order should be the same as the order in df_lst)
    d : int
        the value to be used in calculating the d-day VaR (e.g. 1-day, 5-day)
    p : int
        the value to be used in calculating the p% VaR (e.g. 99, 95)
    ewma_par_lst : list (of float)
        ewma_par_i = the value of the lambda parameter in an exponentially-weighted moving average model 
        for ith stock (order should be the same as the order in df_lst) 
        assumes the value is in range (0,1)
        assumes there is no None value in the list
    covar_ewma_par_mat : numpy.array
        covar_ewma_par_mat[i, j] = the EWMA decay parameter for the covariance of the returns of asset i and j
        this matrix only has value if EWMA will be used
    
    Returns
    -------
    float (2 decimal places)
        diversified d-day p% VaR of a portfolio of stocks using Delta Normal Approach
    """
    # Initialize arrays
    a = [0 for i in range(len(df_lst))] # alpha vector
    return_lst = [0 for i in range(len(df_lst))] # array of returns
    
    # Process each df in df_lst
    for i in range(len(df_lst)):
        
        # Compute a_i
        a[i] = N_lst[i] * df_lst[i].loc[0,"close"]
    
        # Get the returns
        df_lst[i]["previous"] = df_lst[i]["close"].shift(-d)
        df_lst[i]["return"] = df_lst[i].apply(get_return,axis=1)   
        
        # Append the current list of return to return_lst
        return_lst[i] = df_lst[i]["return"][:-d]
    
    # Convert a to a numpy array
    a = np.array(a)
    
    if ewma_par_lst:
        # Initialize Capital Sigma (Covariance Matrix)
        SIGMA = [[0 for i in range(len(df_lst))] for j in range(len(df_lst))]
        
        # Process diagonals (Variance)
        for i in range(len(df_lst)):
            df_lst[i] = get_weights(df_lst[i],d,ewma_par_lst[i])
            SIGMA[i][i] = np.nansum((df_lst[i]["return"]**2) * df_lst[i]["weight"])
        
        # Process off-diagonals (Covariance)
        df_len = len(df_lst[0]["close"])-d
        for i in range(len(df_lst)):
            for j in range(i+1,len(df_lst)):
                weights = np.array([(1-covar_ewma_par_mat[i][j])*(covar_ewma_par_mat[i][j]**k) for k in range(df_len)])
                tmp = df_lst[i]["return"] * df_lst[j]["return"]
                cov_ij = np.nansum(tmp[:-d] * weights)
                SIGMA[i][j] = SIGMA[j][i] = cov_ij

    else:
        # Get the Covariance Matrix
        SIGMA = np.cov(return_lst)
    
    # Compute the standard deviation of the portfolio
    sigma = np.sqrt(np.dot(a.T,np.dot(SIGMA,a)))
    
    quantile = NormalDist().inv_cdf(p/100)
    
    VaR = sigma * quantile
        
    return VaR

## Part 2
In Part 2, you will be using the functions you created in Part 1 to answer the following questions. The stock price data for these items can be found in the `data` folder of our [class Github repository](https://github.com/ateneobsamf2021/ma195l2). The historical data covers the trading period from February 28, 2011 to February 26, 2021.

Assume that today is February 26, 2021. You are a portfolio risk manager who is assigned to analyze the market risk associated to the following portfolios of stocks containing the indicated number of shares per stock:

<table>
    <tr>
        <td>
            Portfolio
        </td>
        <td>
            SM
        </td>
        <td>
            TEL
        </td>
        <td>
            GLO
        </td>
        <td>
            MBT
        </td>
        <td>
            SCC
        </td>  
    </tr>
        <tr>
        <td>
            A
        </td>
        <td>
            1000
        </td>
        <td>
            -
        </td>
        <td>
            -
        </td>
        <td>
            -
        </td>
        <td>
            -
        </td>
    </tr>
        <tr>
        <td>
            B
        </td>
        <td>
            -
        </td>
        <td>
            370
        </td>
        <td>
            630
        </td>
        <td>
            -
        </td>
        <td>
            -
        </td>
    </tr>
        <tr>
        <td>
            C
        </td>
        <td>
            1443
        </td>
        <td>
            154
        </td>
        <td>
            1642
        </td>
        <td>
            2433
        </td>
        <td>
            1750
        </td>  
    </tr>
</table>

The stocks under consideration are SM Investments Corporation (SM), PLDT, Inc. (TEL), Globe Telecom, Inc. (GLO), Metropolitan Bank and Trust Company (MBT), and Semirara Mining Corporation (SCC).

Import the data frames from GitHub

In [6]:
SM = pd.read_csv('https://raw.githubusercontent.com/ateneobsamf2021/ma195l2/main/data/PA1/SM.csv')
TEL = pd.read_csv('https://raw.githubusercontent.com/ateneobsamf2021/ma195l2/main/data/PA1/TEL.csv')
GLO = pd.read_csv('https://raw.githubusercontent.com/ateneobsamf2021/ma195l2/main/data/PA1/GLO.csv')
MBT = pd.read_csv('https://raw.githubusercontent.com/ateneobsamf2021/ma195l2/main/data/PA1/MBT.csv')
SCC = pd.read_csv('https://raw.githubusercontent.com/ateneobsamf2021/ma195l2/main/data/PA1/SCC.csv')

# Reverse order if needed (newest to oldest)
if True:
    SM = SM.loc[::-1].reset_index(drop = True)
    TEL = TEL.loc[::-1].reset_index(drop = True)
    GLO = GLO.loc[::-1].reset_index(drop = True)
    MBT = MBT.loc[::-1].reset_index(drop = True)
    SCC = SCC.loc[::-1].reset_index(drop = True)

## Question A

Suppose market risk metrics are requested for **Portfolio A**.

1. Use the delta-normal approach to determine the portfolio's one-day 99\% VaR.
2. Use the five-day continuously compounded returns and the delta-normal approach to determine the portfolio's five-day 99\% VaR.
3. Assume that the one-day volatility of the stock follows an exponentially-weighted moving average model with parameter $\lambda=0.69$. Determine the portfolio's one-day 99\% VaR.

In [7]:
A1 = undiversified_d_day_p_percent_VaR_delta_normal(df_lst = [SM], N_lst = [1000], d = 1, p = 99, ewma_par_lst=None)
print("The one-day 99% VaR is "+str(round(A1,2))+".")

A2 = undiversified_d_day_p_percent_VaR_delta_normal(df_lst = [SM], N_lst = [1000], d = 5, p = 99, ewma_par_lst=None)
print("The five-day 99% VaR is "+str(round(A2,2))+".")

A3 = undiversified_d_day_p_percent_VaR_delta_normal(df_lst = [SM], N_lst = [1000], d = 1, p = 99, ewma_par_lst=[0.69])
print("The one-day 99% VaR with EWMA parameter 0.69 is "+str(round(A3,2))+".")

The one-day 99% VaR is 1671.6.
The five-day 99% VaR is 3803.83.
The one-day 99% VaR with EWMA parameter 0.69 is 2996.38.


## Question B

We now look at the value-at-risk metrics for **Portfolio B**.

1. Calculate the diversified one-day 97.5\% VaR for the portfolio using the delta-normal approach. What is the benefit of diversification?
2. Use the three-day continuously compounded returns and the delta-normal approach to determine the portfolio's undiversified and diversified three-day 97.5\% VaR.
3. Assume that the three-day volatility of each stock and the covariance of their returns follow exponentially weighted moving average models with the following decay parameters:

<table>
    <tr>
        <td>
           Data
        </td>
        <td>
            Stock A
        </td>
        <td>
            Stock B
        </td>
        <td>
            Covariance
        </td>
    </tr>
        <tr>
        <td>
            Decay Parameter
        </td>
        <td>
            0.92
        </td>
        <td>
            0.88
        </td>
        <td>
            0.77
        </td>
    </tr>
</table>

Determine the diversified and undiversified three-day 97.5\% VaR for the portfolio.

In [8]:
B11 = diversified_d_day_p_percent_VaR_delta_normal(df_lst = [TEL,GLO], N_lst = [370,630], d = 1, p = 97.5, ewma_par_lst=None, covar_ewma_par_mat=None)
B12 = undiversified_d_day_p_percent_VaR_delta_normal(df_lst = [TEL,GLO], N_lst = [370,630], d = 1, p = 97.5, ewma_par_lst=None)
print("The diversified one-day 97.5% VaR is "+str(round(B11))+". The benefit of diversification is "+str(round(B12-B11,2))+".")

B21 = diversified_d_day_p_percent_VaR_delta_normal(df_lst = [TEL,GLO], N_lst = [370,630], d = 3, p = 97.5, ewma_par_lst=None, covar_ewma_par_mat=None)
B22 = undiversified_d_day_p_percent_VaR_delta_normal(df_lst = [TEL,GLO], N_lst = [370,630], d = 3, p = 97.5, ewma_par_lst=None)
print("The diversified three-day 97.5% VaR is "+str(round(B21,2))+". The undiversified VaR is "+str(round(B22,2))+".")

B31 = diversified_d_day_p_percent_VaR_delta_normal(df_lst = [TEL,GLO], N_lst = [370,630], d = 3, p = 97.5, ewma_par_lst=[0.92,0.88], covar_ewma_par_mat = [[0,0.77],[0.77,0]])
B32 = undiversified_d_day_p_percent_VaR_delta_normal(df_lst = [TEL,GLO], N_lst = [370,630], d = 3, p = 97.5, ewma_par_lst=[0.92,0.88])
print("The diversified three-day 97.5% VaR is "+str(round(B31,2))+". The undiversified VaR is "+str(round(B32,2))+".")

The diversified one-day 97.5% VaR is 1703. The benefit of diversification is 61.5.
The diversified three-day 97.5% VaR is 3011.86. The undiversified VaR is 3116.19.
The diversified three-day 97.5% VaR is 2507.32. The undiversified VaR is 2718.44.


## Question C

We now consider the possible losses on **Portfolio C**.

1. Use the delta-normal approach to calculate the undiversified one-day 95\% portfolio VaR. Determine the corresponding undiversified 10-day 95\% VaR under the assumption that returns are independent and identically distributed.
2. Which of stocks in the portfolio has the highest contribution to the undiversified one-day 95\% VaR? Why? Compare the individual VaRs and determine which factor/s led to the stock's VaR contribution.
3. Use the ten-day continuously compounded returns and the delta-normal approach to determine the portfolio's undiversified and diversified 10-day 95\% VaR.

In [9]:
C11 = undiversified_d_day_p_percent_VaR_delta_normal(df_lst = [SM,TEL,GLO,MBT,SCC], N_lst = [1443,154,1642,2433,1750], d = 1, p = 95, ewma_par_lst=None)
C12 = undiversified_d_day_p_percent_VaR_delta_normal(df_lst = [SM,TEL,GLO,MBT,SCC], N_lst = [1443,154,1642,2433,1750], d = 10, p = 95, ewma_par_lst=None)
print("The undiversified one-day 95% VaR is "+str(round(C11,2))+". The undiversified 10-day VaR is "+str(round(C12,2))+".")
print("")

C21 = undiversified_d_day_p_percent_VaR_delta_normal(df_lst = [SM,TEL,GLO,MBT,SCC], N_lst = [1443,0,0,0,0], d = 1, p = 95, ewma_par_lst=None)
print("The undiversified one-day 95% VaR for SM is "+str(round(C21,2))+".")
C22 = undiversified_d_day_p_percent_VaR_delta_normal(df_lst = [SM,TEL,GLO,MBT,SCC], N_lst = [0,154,0,0,0], d = 1, p = 95, ewma_par_lst=None)
print("The undiversified one-day 95% VaR for TEL is "+str(round(C22,2))+".")
C23 = undiversified_d_day_p_percent_VaR_delta_normal(df_lst = [SM,TEL,GLO,MBT,SCC], N_lst = [0,0,1642,0,0], d = 1, p = 95, ewma_par_lst=None)
print("The undiversified one-day 95% VaR for GLO is "+str(round(C23,2))+".")
C24 = undiversified_d_day_p_percent_VaR_delta_normal(df_lst = [SM,TEL,GLO,MBT,SCC], N_lst = [0,0,0,2433,0], d = 1, p = 95, ewma_par_lst=None)
print("The undiversified one-day 95% VaR for MBT is "+str(round(C24,2))+".")
C25 = undiversified_d_day_p_percent_VaR_delta_normal(df_lst = [SM,TEL,GLO,MBT,SCC], N_lst = [0,0,0,0,1750], d = 1, p = 95, ewma_par_lst=None)
print("The undiversified one-day 95% VaR for SCC is "+str(round(C25,2))+".")
print("- SM has the highest contribution to the undiversified one-day 95% VaR due to its high standard deviation of 0.0518.")
print("- The amount of shares and share price did not affect the results since they are very near each other.")
print("")

C31 = undiversified_d_day_p_percent_VaR_delta_normal(df_lst = [SM,TEL,GLO,MBT,SCC], N_lst = [1443,154,1642,2433,1750], d = 10, p = 95, ewma_par_lst=None)
C32 = diversified_d_day_p_percent_VaR_delta_normal(df_lst = [SM,TEL,GLO,MBT,SCC], N_lst = [1443,154,1642,2433,1750], d = 10, p = 95, ewma_par_lst=None, covar_ewma_par_mat=None)
print("The undiversified 10-day 95% VaR is "+str(round(C31,2))+". The diversified 10-day 95% VaR is "+str(round(C32,2))+".")

The undiversified one-day 95% VaR is 4143.87. The undiversified 10-day VaR is 13357.1.

The undiversified one-day 95% VaR for SM is 1705.5.
The undiversified one-day 95% VaR for TEL is 551.82.
The undiversified one-day 95% VaR for GLO is 404.32.
The undiversified one-day 95% VaR for MBT is 790.57.
The undiversified one-day 95% VaR for SCC is 691.67.
- SM has the highest contribution to the undiversified one-day 95% VaR due to its high standard deviation of 0.0518.
- The amount of shares and share price did not affect the results since they are very near each other.

The undiversified 10-day 95% VaR is 13357.1. The diversified 10-day 95% VaR is 7995.69.
